In [2]:
from config import db_config
from collections import defaultdict
import pyodbc
from beem.account import Account

In [3]:
connection = pyodbc.connect(driver=db_config['driver'],
                            server=db_config['server'],
                            database=db_config['database'],
                            uid=db_config['uid'],
                            pwd=db_config['pwd'])

In [4]:
cursor = connection.cursor()

In [5]:
query = """\
SELECT TOP 100 account
FROM (
       SELECT SUBSTRING(body, CHARINDEX('@', body) + 1, CHARINDEX(' leads', body) - CHARINDEX('@', body) - 1) as account, *
       FROM Comments (NOLOCK)
       WHERE depth > 0 AND
             author = 'guard' AND CONTAINS(body, 'phishing')) C
GROUP BY account
ORDER BY COUNT(*) DESC"""

with open('scammers.txt', 'w') as f:
    f.write('\n'.join(row[0] for row in cursor.execute(query)))

In [6]:
query = """\
SELECT TOP 100 author
FROM Comments (NOLOCK)
WHERE depth = 0 AND
      category in ('utopian-io', 'dtube', 'dlive')
ORDER BY NEWID()"""

with open('content_creators.txt', 'w') as f:
    f.write('\n'.join(row[0] for row in cursor.execute(query)))

TypeError: write() argument must be str, not generator

In [ ]:
query = """\
SELECT TOP 100 author
FROM Comments (NOLOCK)
WHERE depth = 1 AND
      created BETWEEN GETUTCDATE() - 90 AND GETUTCDATE() AND
      ((CONTAINS (body, 'nice') AND body LIKE 'nice') OR
       (CONTAINS (body, 'beautiful') AND body LIKE 'beautiful') OR
       (CONTAINS (body, 'upvoted') AND body LIKE 'upvoted'))
GROUP BY author
ORDER BY COUNT(*) DESC"""

with open('comment_spammers.txt', 'w') as f:
    f.write('\n'.join(row[0] for row in cursor.execute(query)))

In [ ]:
d = defaultdict(dict)

for name in ['jacekw', 'fervi', 'gtg', 'noisy', 'tipu', 'gizem', 'moyse']:
    account = Account(name)
    foll = account.get_follow_count()
    d[name]['followers'] = foll['follower_count']
    d[name]['followings'] = foll['following_count']
    d[name]['follow ratio'] = foll['following_count'] / foll['follower_count']
    d[name]['muters'] = len(account.get_muters())
    d[name]['reputation'] = account.get_reputation()
    d[name]['effective sp'] = account.get_steem_power()
    d[name]['own sp'] = account.get_steem_power(onlyOwnSP=True)
    d[name]['sp ratio'] = account.get_steem_power() / account.get_steem_power(onlyOwnSP=True)

In [ ]:
query = """\
SELECT
  name,
  curation_rewards,
  posting_rewards,
  witnesses_voted_for
FROM Accounts (NOLOCK) a
WHERE name in ('jacekw', 'fervi', 'gtg', 'noisy', 'tipu', 'gizem', 'moyse')"""

for row in cursor.execute(query):
    name = row[0]
    curation_rewards = row[1] / 1000.0
    posting_rewards = row[2] / 1000.0
    witnesses_voted_for = row[3]   
    d[name]['curation_rewards'] = curation_rewards
    d[name]['posting_rewards'] = posting_rewards
    d[name]['witnesses_voted_for'] = witnesses_voted_for

In [ ]:
for k, v in d.items():
    print(k)
    for k2, v2 in v.items():
        print('\t', k2, ':', v2)

In [ ]:
query = """\
SELECT
  author,
  COUNT(*),
  AVG(LEN(body))
FROM Comments (NOLOCK) a
WHERE depth = 0 AND
      created BETWEEN GETUTCDATE() - 90 AND GETUTCDATE()
      AND author in ('jacekw', 'fervi', 'gtg', 'noisy', 'tipu', 'gizem', 'moyse')
GROUP BY author"""

In [ ]:
for row in cursor.execute(query):
    name = row[0]
    posts = row[1]
    average_post_len = row[2]
    d[name]['posts'] = posts
    d[name]['average_post_len'] = average_post_len